In [1]:
#%matplotlib widget
import collections
import backtrader as bt
from backtradermql5.mt5store import MTraderStore
from datetime import datetime, timedelta
from local.ind_supertrend import SuperTrend
import os

In [2]:
class MyStrategy(bt.Strategy):
    params = (
        ('mt5broker', False),
        ('stperiod', 7),
        ('stmultiplier', 3),
        ('doprint', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if doprint or self.params.doprint:
            #dt = dt or self.datas[0].datetime.datetime()
            dt = dt or self.data.datetime.datetime()
            print(f'{dt}: {txt}')

    def __init__(self):
        self.dataclose = self.datas[0].close
        #self.data1close = self.datas[1].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.inBuyPosition = False
        self.inSellPosition = False
        self.live_data = False
        self.last2ST = 0
        #self.activetradeid = None
        #global buy_order 
        #self.bar_executed = 0

        super().__init__()
        #self.broker.set_coc(True)

        self.ma1 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=3)
            #self.datas[1], period=self.params.ma1period)

    def notify_order(self, order):

        if order.status in [order.Submitted]:
            self.log('Order submitted')
            return
        if order.status in [order.Accepted]:
            self.log('Order accepted')
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.5f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price, order.executed.value,
                          order.executed.comm))

            elif order.issell():
                self.log('SELL EXECUTED, Price: %.5f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price, order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)
            self.bar_executed2 = 0 # reset for higher tf

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            # capture the trade id
            self.log('Notify trade: a new position')
            return

        self.log('Notify trade (closed): OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # get pos and dpos parameter
        pos = self.getposition(self.data)
        dpos = pos.size
        #self.log('Close, %.5f, Pos size: %.5f' % (self.dataclose[0], dpos))

        cash = self.broker.getcash()
        #if self.live_data:
        #    cash = self.broker.getcash()
        #else
        #    cash = 'NA'

        # for multiple data
        for data in self.datas:
            data = self.data
            self.log(f'{data._name} | Cash {cash} | O: {data.open[0]} H: {data.high[0]} L: {data.low[0]} C: {data.close[0]} V:{data.volume[0]} Pos:{dpos}')
        #data = self.data
        #self.log(f'{data._name} | Cash {cash} | O: {data.open[0]} H: {data.high[0]} L: {data.low[0]} C: {data.close[0]} V:{data.volume[0]} Pos:{dpos}')
        
        # Check if we are in the market
        #if not self.position:
        ''' Test strategy '''
        #### simple strategi from cerebro (just for test.. )

        if not dpos:
            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.dataclose[-1]:
                    # current close less than previous close

                # BUY, BUY, BUY!!! (with default parameters)
                self.log('BUY CREATE, %.5f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                if self.live_data or not self.params.mt5broker: 
                    self.order = self.buy() 
                    self.buy_order = self.order

        else:
            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 2):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.5f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                # self.order = self.sell()
                if self.live_data: 
                    self.cancel(self.buy_order) # self.trade.tradeid)
                elif not self.params.mt5broker:
                    self.order = self.close() # self.trade.tradeid)
    
        '''
        ####
        if not dpos:
            # Not yet ... we MIGHT BUY if
            if (self.stcross < 0) or ((self.data1close[0] > self.last2ST)
                                      and self.last2ST != 0):
                self.log('BUY CREATE1, %.5f' % self.data1close[0])
                if self.live_data or not self.params.mt5broker: 
                    #self.order = self.buy()
                    self.order = self.buy_bracket(exectype=bt.Order.Market, stopprice=1.32724)
                self.inBuyPosition = True

            # if supertrend  is crossing..or
            if (self.stcross > 0) or ((self.data1close[0] < self.last2ST)
                                      and self.last2ST != 0):
                self.log('SELL CREATE1, %.5f' % self.data1close[0])
                if self.live_data or not self.params.mt5broker: 
                    self.order = self.sell()
                    #self.order = self.sell_bracket(exectype=bt.Order.Market, stopprice=self.last2ST)
                self.inSellPosition = True
            
        else:
            # Already in the market ..  # this can be simplified .. no need inbuy/insell position now.. since we have .close()
            if self.inBuyPosition == True:
                # close position if..
                if self.stcross > 0 and dpos >= 0:
                    # SELL, SELL, SELL!!! (with all possible default parameters)
                    self.log('SELL CREATE2, %.5f' % self.data1close[0])

                    if self.live_data: 
                        self.order = self.cancel(self.order) # self.trade.tradeid)
                    elif not self.params.mt5broker:
                        self.order = self.close() # self.trade.tradeid)

                    #if self.live_data or not self.params.mt5broker: 
                        #self.order = self.close(tradeid=self.activetradeid) # self.trade.tradeid)

                    self.inBuyPosition = False
                    self.last2ST = self.x[0]  

            if self.inSellPosition == True:
                # close position if..
                if self.stcross < 0 and dpos <= 0:
                    # BUY!!! (with all possible default parameters)
                    self.log('BUY CREATE2, %.5f' % self.data1close[0])

                    if self.live_data: 
                        self.order = self.cancel(self.order) # self.trade.tradeid)
                    elif not self.params.mt5broker:
                        self.order = self.close() # self.trade.tradeid)

                    #if self.live_data or not self.params.mt5broker: 
                        #self.order = self.close(tradeid=self.activetradeid) #self.trade.tradeid)
                    #    self.order = self.cancel(self.order) # self.trade.tradeid)

                    self.inSellPosition = False
                    self.last2ST = self.x[0]  
    '''
    def notify_data(self, data, status, *args, **kwargs):
        dn = data._name
        dt = datetime.now()

        self.log('Data Status: %s' % data._getstatusname(status), dt=dt)
        if data._getstatusname(status) == 'LIVE':
            self.live_data = True
        else:
            self.live_data = False

#    def notify_store(self, msg): # is it causing the trouble?
#        self.log('Notify store: %s' % msg)
        
    def stop(self):
        self.log('(ST (period,mul): %2d, %2d) Ending Value %.2f'
            % (self.params.stperiod, self.params.stmultiplier, self.broker.getvalue()), 
                 doprint=True)
    

In [3]:
def runstrat(ppair='EURUSD',
             ptf=bt.TimeFrame.Minutes,
             pcomp=None,
             ptf1=bt.TimeFrame.Minutes,
             pcomp1=30,
             preplay=True,
             pstart_date="2020-03-21",
             pend_date="2020-03-22",
             pwrite_csv=False,
             pread_csv=False,
             phost='192.168.100.110',
             pdebug=False,
             phistory=False,
             pmt5broker=False,
             psizer_type=bt.sizers.FixedSize,
             pcash=10000,
             pstake=1,
             pcommission=15,
             pmargin=1000,
             pmult=100000,
             pplot=True,
             panalyze=True,
             pstrategy=MyStrategy,
             pstperiod=7,
             pstmultiplier=3,
             pdoprint=False,
             p_usepositions=True,
             optimize=False):
    #             pma1period=5,
    #             pma2period=10,
    #             pma3period=15,
    #             patrperiod=20,
    to_str = collections.OrderedDict((
        (bt.TimeFrame.Ticks, 'T'),
        (bt.TimeFrame.Minutes, 'M'),
        (bt.TimeFrame.Days, 'Day'),
        (bt.TimeFrame.Weeks, 'W'),
        (bt.TimeFrame.Months, 'M'),
        (bt.TimeFrame.Years, 'Y'),
    ))

    cerebro = bt.Cerebro()
    store = MTraderStore(host=phost, debug=pdebug)

    if pmt5broker:
        broker = store.getbroker(use_positions=p_usepositions)  #
        cerebro.setbroker(broker)

        data = store.getdata(dataname=ppair,
                             timeframe=ptf,
                             compression=pcomp,
                             fromdate=pstart_date,
                             historical=False)
    else:
        if pwrite_csv:
            store.write_csv(symbol=ppair,
                            timeframe=ptf,
                            compression=pcomp,
                            fromdate=pstart_date,
                            todate=pend_date)
            # convert to utf-8
            cmd = f'iconv -f UTF-16 -t UTF-8 -o /home/awahyudi/Downloads/datas/{ppair}-{to_str[ptf]}{pcomp}-utf8.csv /media/winshare/{ppair}-{to_str[ptf]}{pcomp}.csv'
            os.system(cmd)

#2020.03.27 19:10:00
#dtformat=('%Y.%m.%d %H:%M:%S\t'),
#                tmformat=('%H:%M:%S\t'),
#dtformat=('%Y.%m.%d'),
#            dataname='/media/winshare/EURUSD-M5-utf8.csv',
#            dataname='/home/awahyudi/Projects/EURUSD-utf.8',

    if pread_csv:
        data = bt.feeds.GenericCSVData(
            dataname=
            f'/home/awahyudi/Downloads/datas/{ppair}-{to_str[ptf]}{pcomp}-utf8.csv',
            compression=1,
            timeframe=ptf,
            fromdate=pstart_date,
            todate=pend_date,
            dtformat=('%Y.%m.%d %H:%M:%S\t'),
            nullvalue=0.0,
            datetime=0,
            open=1,
            high=2,
            low=3,
            close=4,
            volume=5,
            openinterest=-1)

#            data = store.getdata(dataname=ppair,
#                         timeframe=ptf,
#                         compression=pcomp,
#                         fromdate=pstart_date,
#                         todate=pend_date,
#                         historical=True)
    elif not pmt5broker:
        data = store.getdata(dataname=ppair,
                             timeframe=ptf,
                             compression=pcomp,
                             fromdate=pstart_date,
                             todate=pend_date,
                             historical=True)


#                             - timedelta(hours=5),

# Add the Data Feed to Cerebro
#cerebro.adddata(data)

    if pcomp1:
        if preplay:
            cerebro.replaydata(data, timeframe=ptf1, compression=pcomp1)
        else:
            cerebro.resampledata(data, timeframe=ptf1, compression=pcomp1)

    # set sizer
    cerebro.addsizer(psizer_type, stake=pstake)

    #----- Normal Strategy
    #cerebro.addstrategy(MyStrategy, ma1period=5, ma2period=10, ma3period=17, atrperiod=10)
    #    pstrategy,
    if optimize == True:
        strats = cerebro.optstrategy(pstrategy,
                                     stperiod=range(5, 12),
                                     stmultiplier=range(5, 12))
        #stperiod=range(5,9),
        #stmultiplier=range(6,10))
    else:
        cerebro.addstrategy(pstrategy,
                            stperiod=pstperiod,
                            stmultiplier=pstmultiplier,
                            mt5broker=pmt5broker,
                            doprint=pdoprint)
    #----- Set commission
    cerebro.broker.setcommission(commission=pcommission,
                                 margin=pmargin,
                                 mult=pmult)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    if not optimize:
        results = cerebro.run()
    elif pread_csv:
        results = cerebro.run(maxcpus=4, stdstats=False)
    else:
        results = cerebro.run(maxcpus=1, stdstats=False)

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    if pplot and not (optimize):
        cerebro.plot(style='candlestick', volume=True)

In [ ]:
%matplotlib widget
#         ptf1=bt.TimeFrame.Minutes,
#         pcomp1=240,
#         pstart_date=1585047600,
#         phistory=False,
#         pstart_date=datetime.now() - timedelta(hours=5+60),
    #end_date=datetime(2019,3,1),
    #start_date = datetime(2019,1,1)
         #pstart_date=datetime("2020-03-27 19:10:00"),
         #pstart_date=datetime(2020,3,27,19,10,0) - timedelta(hours=5),

runstrat(ppair='GBPUSD',
         ptf=bt.TimeFrame.Minutes,
         pcomp=1,
         ptf1=bt.TimeFrame.Minutes,
         pcomp1=1,
         preplay=False, 
         pstart_date=datetime.now() - timedelta(hours=4) - timedelta(minutes=2),
         pend_date=datetime.now() - timedelta(hours=4),
         pread_csv=False,
         pwrite_csv=False,
         phost='192.168.100.113',
         pdebug=True,
         pmt5broker=True,
         pplot=True,
         pdoprint = True,
         p_usepositions = False,
         psizer_type=bt.sizers.FixedSize,
         pcash=10000,
         pstake=1,
         pcommission=15,
         pmargin=1000,
         pmult=100000,
         panalyze=False,
         pstrategy=MyStrategy,
         pstperiod=5,
         pstmultiplier=9,
         optimize = False
        ) 
         #pstperiod=6, 10
         #pstmultiplier=9, 11
         #pma1period=5,
         #pma2period=9,
         #pma3period=15,
         #pstrategy=superTrendStrategy,
         #pstrategy=MyStrategy, (don't forget to enable the xxxperiod param in addStrategy, FIX this!)

            # tf=1, period=3,multiplier=3
            

ZMQ SYS REQUEST:  {'action': 'RESET', 'actionType': None, 'symbol': None, 'chartTF': None, 'fromDate': None, 'toDate': None, 'id': None, 'magic': None, 'volume': None, 'price': None, 'stoploss': None, 'takeprofit': None, 'expiration': None, 'deviation': None, 'comment': None}  ->  OK
ZMQ DATA REPLY:  {'error': False, 'lastError': '0', 'description': 'ERR_SUCCESS', 'function': 'ResetSubscriptions'}
Starting Portfolio Value: 0.00
ZMQ SYS REQUEST:  {'action': 'ACCOUNT', 'actionType': None, 'symbol': None, 'chartTF': None, 'fromDate': None, 'toDate': None, 'id': None, 'magic': None, 'volume': None, 'price': None, 'stoploss': None, 'takeprofit': None, 'expiration': None, 'deviation': None, 'comment': None}  ->  OK
ZMQ DATA REPLY:  {'error': False, 'broker': 'FBS Inc', 'currency': 'USD', 'server': 'FBS-Demo', 'trading_allowed': 1, 'bot_trading': 1, 'balance': 9812.0, 'equity': 9772.0, 'margin': 551.4, 'margin_free': 9220.6, 'margin_level': 1772.21618}
error - False
broker - FBS Inc
currency 

ZMQ DATA REPLY:  {'error': False, 'retcode': 10009, 'desription': 'TRADE_RETCODE_DONE', 'order': 40672169, 'volume': 1.0, 'price': 1.242, 'bid': 0.0, 'ask': 0.0, 'function': 'TradingModule'}
{'error': False, 'retcode': 10009, 'desription': 'TRADE_RETCODE_DONE', 'order': 40672169, 'volume': 1.0, 'price': 1.242, 'bid': 0.0, 'ask': 0.0, 'function': 'TradingModule'}ZMQ STREAMING TRANSACTION: 
 {'request': {'action': 'TRADE_ACTION_DEAL', 'order': 40672169, 'symbol': 'GBPUSD', 'volume': 1.0, 'price': 0.0, 'stoplimit': 0.0, 'sl': 0.0, 'tp': 0.0, 'deviation': 10, 'type': 'ORDER_TYPE_BUY', 'type_filling': 'ORDER_FILLING_FOK', 'type_time': 'ORDER_TIME_GTC', 'expiration': 0, 'comment': None, 'position': 0, 'position_by': 0}, 'result': {'retcode': 10009, 'result': 'TRADE_RETCODE_DONE', 'deal': 40672169, 'order': 40672169, 'volume': 1.0, 'price': 1.242, 'comment': None, 'request_id': 4, 'retcode_external': 0}}
{'action': 'TRADE_ACTION_DEAL', 'order': 40672169, 'symbol': 'GBPUSD', 'volume': 1.0, 'pr

ZMQ LIVE DATA:  {'status': 'CONNECTED', 'symbol': 'GBPUSD', 'timeframe': 'M1', 'data': [1585711560, 1.24228, 1.24228, 1.24192, 1.24218, 18.0]}
2020-04-01 03:26:00: Order Canceled/Margin/Rejected
2020-04-01 03:26:00: GBPUSD | Cash 9812.0 | O: 1.24228 H: 1.24228 L: 1.24192 C: 1.24218 V:18.0 Pos:0.0


# EURUSD 
# 30min timeframe
period, mult => 6,9
# day 20-0
Starting Portfolio Value: 10000.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  5,  6) Ending Value 14776.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  5,  7) Ending Value 15308.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  5,  8) Ending Value 14970.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  5,  9) Ending Value 14588.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  6,  6) Ending Value 15495.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  6,  7) Ending Value 15349.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  6,  8) Ending Value 14970.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  6,  9) Ending Value 15398.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  7,  6) Ending Value 15511.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  7,  7) Ending Value 14970.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  7,  8) Ending Value 15780.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  7,  9) Ending Value 15196.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  8,  6) Ending Value 15398.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  8,  7) Ending Value 15780.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  8,  8) Ending Value 15398.00
2020-03-23 15:00:00: (SMA period  1,  1,  1: ST (period,mul):  8,  9) Ending Value 15196.00
Final Portfolio Value: 15196.00

# day 40-20
Starting Portfolio Value: 10000.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  5,  6) Ending Value 11012.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  5,  7) Ending Value 11907.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  5,  8) Ending Value 12983.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  5,  9) Ending Value 13299.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  6,  6) Ending Value 11796.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  6,  7) Ending Value 13059.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  6,  8) Ending Value 13516.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  6,  9) Ending Value 13299.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  7,  6) Ending Value 11632.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  7,  7) Ending Value 13502.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  7,  8) Ending Value 13299.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  7,  9) Ending Value 13299.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  8,  6) Ending Value 12665.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  8,  7) Ending Value 13502.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  8,  8) Ending Value 13299.00
2020-03-03 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  8,  9) Ending Value 13299.00
Final Portfolio Value: 13299.00
    
# day 60-40
Starting Portfolio Value: 10000.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  5,  6) Ending Value 12261.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  5,  7) Ending Value 11782.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  5,  8) Ending Value 11782.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  5,  9) Ending Value 11782.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  6,  6) Ending Value 12261.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  6,  7) Ending Value 11782.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  6,  8) Ending Value 11782.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  6,  9) Ending Value 11738.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  7,  6) Ending Value 11890.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  7,  7) Ending Value 11782.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  7,  8) Ending Value 11782.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  7,  9) Ending Value 11738.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  8,  6) Ending Value 11890.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  8,  7) Ending Value 11782.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  8,  8) Ending Value 11738.00
2020-02-12 20:00:00: (SMA period  1,  1,  1: ST (period,mul):  8,  9) Ending Value 11715.00
Final Portfolio Value: 11715.00

